In [5]:
import sys
sys.path.append('quantum_annealing/')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

import plot_boundary
import QUBO_SVM_functions as qSVM
import kernel_SVM_functions as kSVM
import metric_functions as mf
from kernel_SVM_functions import rbf_kernel
from parameter_tuning import tune_csvm_parameters

from quantum_classifier import QSVMq
from quantum_annealing_functions import QA_cross_validate

In [6]:
filepath = 'synth_data/synth_0.3.csv'

data = np.loadtxt(filepath, delimiter = ',')
X = data[:, :-1]
t = data[:, -1]

X_train, X_test, t_train, t_test = train_test_split(X, t, train_size = 0.17, shuffle = True, stratify = t)
t_train = t_train.reshape(-1, 1)
t_test = t_test.reshape(-1, 1)
print(X_train.shape)

(20, 2)


In [7]:
df = pd.read_csv('QA_results/testing_samples.csv')
df

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,29,30,31,32,33,34,35,chain_break_fraction,energy,num_occurrences
0,0,0,0,0,1,1,1,0,1,1,...,1,0,0,0,1,1,1,0.000000,-741.819521,95
1,1,0,0,0,1,1,1,1,1,1,...,1,0,0,0,1,1,1,0.027778,-741.819521,1
2,2,0,0,0,1,1,1,1,1,1,...,1,0,0,0,1,1,1,0.027778,-741.819521,1
3,3,0,0,0,1,1,1,1,1,1,...,1,0,0,0,1,1,1,0.027778,-741.819521,1
4,4,0,0,0,1,1,1,1,1,1,...,1,0,0,0,1,1,1,0.027778,-741.819521,1
5,5,0,0,0,1,1,1,1,1,1,...,1,0,0,0,1,1,1,0.027778,-741.819521,1


In [8]:
clf = QSVMq(2, 3, 1, rbf_kernel, 0.125)

In [10]:
QA_cross_validate(X_train, t_train, classifier = clf, filepath = 'quantum_analysis')

[[ 1.]
 [-1.]]
[[ 1.]
 [-1.]]
[[ 1.]
 [-1.]]


[[ 1.]
 [-1.]]
[[-1.]
 [ 1.]]
[[ 1.]
 [-1.]]
[[-1.]
 [ 1.]]
[[-1.]
 [ 1.]]
[[-1.]
 [ 1.]]
[[-1.]
 [ 1.]]


([1.0, 0.2, 0.9, 0.5, 1.0, 0.8, 0.7, 0.4, 0.6, 1.0],
 [0.55, 0.45, 0.6, 0.5, 0.6, 0.55, 0.45, 0.45, 0.5, 0.55])